In [33]:
# Import auth client library

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [34]:
! pip install osmnx

In [35]:
# Libraries

import pandas as pd
import scipy.spatial
import numpy as np
from google.cloud import bigquery
from geopy.distance import vincenty

import networkx as nx
import osmnx as ox

# Create client instance

client = bigquery.Client(project='uip-production')

In [36]:
# Data import via SQL query

sql = """SELECT
  distinct(dock_group_title),
  dock_group_coords.latitude,
  dock_group_coords.longitude
  
FROM
  `uip-production.bikesharing_GB_edinburghcitybikes.dockgroup_snapshots`"""

In [37]:
# DataFrame 

df = client.query(sql).to_dataframe()

In [38]:
df.dock_group_title.nunique()

166

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dock_group_title  230 non-null    object 
 1   latitude          230 non-null    float64
 2   longitude         230 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.5+ KB


In [40]:
# Clean up

df.drop_duplicates(subset='dock_group_title', inplace=True)

In [41]:
# Calculate distance with scipy.spatial
# Output numpy array


mat = scipy.spatial.distance.cdist(df[['latitude','longitude']], 
                              df[['latitude','longitude']], metric='euclidean')


In [42]:
mat

array([[0.        , 0.07135352, 0.02763719, ..., 0.20500329, 0.01867135,
        0.03941729],
       [0.07135352, 0.        , 0.09665723, ..., 0.13520267, 0.0751363 ,
        0.09874763],
       [0.02763719, 0.09665723, 0.        , ..., 0.22754355, 0.02369578,
        0.02013433],
       ...,
       [0.20500329, 0.13520267, 0.22754355, ..., 0.        , 0.20413757,
        0.22384894],
       [0.01867135, 0.0751363 , 0.02369578, ..., 0.20413757, 0.        ,
        0.0248214 ],
       [0.03941729, 0.09874763, 0.02013433, ..., 0.22384894, 0.0248214 ,
        0.        ]])

In [43]:
# Create DataFrame with array

new_df = pd.DataFrame(mat, index=df['dock_group_title'], columns=df['dock_group_title']) 


In [44]:
new_df

dock_group_title,Colinton Road,St. John's Road 1,West Crosscauseway,George Square,Simon Square,Heriot Watt - Student Accomodation (Anna MacLeod Halls),Gorgie Road,Edinburgh Park Central,Musselburgh Brunton Hall,Dudley Gardens,Kings Buildings 1,Henderson Row,Morningside Road,Pollock Halls,Drumsheugh Place,Milton Road - Edinburgh College,EICC,Edinburgh Zoo,Ellersly Road,South Trinity Road,Queen Margaret University,Brandon Street,Stockbridge,Craigleith Road,Brunswick Place,Cramond Foreshore,Boroughmuir,Constitution Street,RHC - Edinburgh Festival Camping (05th to 26th August),Picardy Place,Dynamic Earth,Meadows - Edinburgh Climate Festival,Edinburgh Park Station,Heriot Watt - Edinburgh Business School,Comely Bank Road,Whitehouse Road,Murrayfield,Eden Locke - Aparthotel (RESIDENTS ONLY),HSBC UK Lets Ride - Meadows Event,Castle Street,...,Haymarket Terrace,Lothian Road,Portobello - Kings Road,Forth Bridge Visitors Centre,Royal Highland Show - West Gate (19th to 23rd June),Portobello,Picady Place,Belford Road,Sustrans - walk cycle event,Kings Buildings 4,Gladstone Terrace,Fort Kinnaird,Hawes Pier,Depot,Causewayside,Western General,Dean Street,Depot Virtual,Meadow Place 2,Shrubhill,Haymarket - Murrayfield Rugby Event,Tesco Ferrymuir,Royal Infirmary,Corstorphine Road - Pinkhill,Lauriston Place,Meggetland,McDonald Road,Meadows East,Kings Building 3,Pollock Halls Virtual,Scotland Street,Waverley Court,Scotstoun House,Bruntsfield,Bruntsfield Links,Waverley Station,Surgeons Hall,Port Edgar Marina,Charlotte Square,Macdonald Road
dock_group_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Colinton Road,0.000000,0.071354,0.027637,0.024244,0.030146,0.120773,0.019677,0.103651,0.152819,0.046184,0.038262,0.027115,0.006142,0.039522,0.017628,0.112682,0.012012,0.059039,0.041563,0.037399,0.137773,0.028510,0.024683,0.035200,0.040103,0.100929,0.007549,0.060417,0.170931,0.033305,0.039005,0.021875,0.096868,0.112211,0.025833,0.099197,0.034585,0.042502,0.020627,0.019056,...,0.017148,0.014292,0.095505,0.200285,0.170931,0.096683,0.033305,0.023591,0.014462,0.035974,0.026080,0.105766,0.183696,0.067019,0.030782,0.036743,0.023503,0.067039,0.017312,0.042356,0.014462,0.196931,0.077904,0.055213,0.017278,0.023760,0.039453,0.028652,0.036958,0.039207,0.030482,0.031971,0.189352,0.005370,0.005352,0.029799,0.028293,0.205003,0.018671,0.039417
St. John's Road 1,0.071354,0.000000,0.096657,0.093313,0.098975,0.058373,0.051869,0.034775,0.223319,0.096179,0.109378,0.079660,0.073384,0.109908,0.069832,0.183060,0.071214,0.012352,0.030827,0.079009,0.208857,0.082274,0.075235,0.045964,0.102253,0.041855,0.066341,0.119381,0.100005,0.096367,0.106227,0.091157,0.030045,0.051336,0.068035,0.030863,0.038005,0.113835,0.090636,0.078926,...,0.058693,0.076001,0.163994,0.130155,0.100005,0.165351,0.096367,0.056260,0.062003,0.106756,0.096720,0.176784,0.114428,0.129232,0.101704,0.053026,0.068888,0.129273,0.087019,0.104050,0.062003,0.126561,0.149257,0.016243,0.084381,0.050291,0.098768,0.098928,0.107971,0.109541,0.087939,0.097858,0.118944,0.075395,0.075401,0.094615,0.096227,0.135203,0.075136,0.098748
West Crosscauseway,0.027637,0.096657,0.000000,0.003428,0.002512,0.148159,0.045228,0.130063,0.126665,0.032686,0.022612,0.025053,0.029418,0.013755,0.028453,0.086405,0.025606,0.084333,0.065920,0.035817,0.112560,0.023691,0.027381,0.054349,0.017451,0.121193,0.030599,0.036432,0.196660,0.012737,0.011737,0.005772,0.123574,0.139641,0.034400,0.122423,0.058708,0.023185,0.007233,0.019749,...,0.038115,0.021094,0.068106,0.223138,0.196660,0.069341,0.012737,0.041944,0.034744,0.023303,0.005899,0.080496,0.205981,0.040767,0.008879,0.051696,0.032237,0.040777,0.010325,0.019468,0.034744,0.220028,0.055056,0.080426,0.012327,0.051346,0.020170,0.004627,0.022525,0.013335,0.019572,0.007912,0.212527,0.022486,0.022493,0.009143,0.002826,0.227544,0.023696,0.020134
George Square,0.024244,0.093313,0.003428,0.000000,0.005916,0.144731,0.041824,0.126665,0.130007,0.032977,0.023808,0.023204,0.026213,0.0168

In [45]:
# create array with minimum distance between points

closest = np.where(new_df.eq(new_df[new_df != 0].min(),0),new_df.columns,False)

# Remove false from the array and get the column names as list . 
df['close'] = [i[i.astype(bool)].tolist() for i in closest]

In [46]:
# Export draft

df.to_excel('ech_coord.xlsx')

In [47]:
df

,dock_group_title,latitude,longitude,close
0,Colinton Road,55.934035,-3.210803,[Bruntsfield Links]
1,St. John's Road 1,55.942746,-3.281623,[St. John's Road 2]
2,West Crosscauseway,55.943862,-3.184972,[Crichton Street]
3,George Square,55.943084,-3.188311,[Crichton Street]
4,Simon Square,55.944868,-3.182671,[West Crosscauseway]
...,...,...,...,...
223,Waverley Station,55.952641,-3.187527,[The Tron]
225,Surgeons Hall,55.946643,-3.185475,[Crichton Street]
226,Port Edgar Marina,55.992957,-3.407156,[Hopetoun Road]
227,Charlotte Square,55.952335,-3.207101,[Castle Street]


In [48]:


df1 = df[['close']]
df1['close'] = df1['close'].agg(lambda x: ','.join(map(str, x)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
df2 = df1.merge(df, left_on='close', right_on='dock_group_title')
df2= df2[['close_x','latitude','longitude']]

In [50]:
df

,dock_group_title,latitude,longitude,close
0,Colinton Road,55.934035,-3.210803,[Bruntsfield Links]
1,St. John's Road 1,55.942746,-3.281623,[St. John's Road 2]
2,West Crosscauseway,55.943862,-3.184972,[Crichton Street]
3,George Square,55.943084,-3.188311,[Crichton Street]
4,Simon Square,55.944868,-3.182671,[West Crosscauseway]
...,...,...,...,...
223,Waverley Station,55.952641,-3.187527,[The Tron]
225,Surgeons Hall,55.946643,-3.185475,[Crichton Street]
226,Port Edgar Marina,55.992957,-3.407156,[Hopetoun Road]
227,Charlotte Square,55.952335,-3.207101,[Castle Street]


In [51]:
df['close'] = df['close'].agg(lambda x: ','.join(map(str, x)))


In [52]:
df3 = df.merge(df2, left_on='close', right_on='close_x', how='outer')
df3 = df3[['dock_group_title','latitude_x','longitude_x','close','latitude_y','longitude_y']]

In [53]:
df3.dropna(inplace=True)

In [54]:
def distance_calc (row):
    start = (row['latitude_x'], row['longitude_x'])
    stop = (row['latitude_y'], row['longitude_y'])

    return vincenty(start, stop).meters

In [55]:
df3['distance'] = df3.apply(lambda row: distance_calc (row),axis=1)


In [56]:
df3

,dock_group_title,latitude_x,longitude_x,close,latitude_y,longitude_y,distance
0,Colinton Road,55.934035,-3.210803,Bruntsfield Links,55.937123,-3.206432,439.181883
1,Colinton Road,55.934035,-3.210803,Bruntsfield Links,55.937123,-3.206432,439.181883
2,Colinton Road,55.934035,-3.210803,Bruntsfield Links,55.937123,-3.206432,439.181883
3,Bruntsfield links,55.937159,-3.206435,Bruntsfield Links,55.937123,-3.206432,3.955444
4,Bruntsfield links,55.937159,-3.206435,Bruntsfield Links,55.937123,-3.206432,3.955444
...,...,...,...,...,...,...,...
272,Meadows East,55.939809,-3.182739,Gladstone Terrace,55.937963,-3.185021,250.189667
273,Waverley Court,55.951734,-3.184179,St Andrews House,55.953164,-3.181682,222.845987
274,Scotstoun House,55.981107,-3.394211,Tesco Ferrymuir,55.983766,-3.401352,535.114797
276,Charlotte Square,55.952335,-3.207101,Castle Street,55.951501,-3.203184,261.766730


In [57]:
df3.drop_duplicates(subset='dock_group_title', inplace=True)

In [58]:
df3.to_excel('ech_coord_dist.xlsx')

In [59]:
G = ox.graph_from_place('Edinburgh, UK', network_type='walk')


df3['orig_node'] = ox.get_nearest_nodes(G, X=df3.longitude_x, Y=df3.latitude_x)
df3['dest_node'] = ox.get_nearest_nodes(G, X=df3.longitude_y, Y=df3.latitude_y)

# how long is our route in meters?
def short_path_length(row):
    return nx.shortest_path_length(G, row['orig_node'], row['dest_node'], weight='length')

In [60]:
df3['short_path_length'] = df3.apply(short_path_length, axis=1)


In [62]:
df3

,dock_group_title,latitude_x,longitude_x,close,latitude_y,longitude_y,distance,orig_node,dest_node,short_path_length
0,Colinton Road,55.934035,-3.210803,Bruntsfield Links,55.937123,-3.206432,439.181883,2290408447,6634566443,491.052
3,Bruntsfield links,55.937159,-3.206435,Bruntsfield Links,55.937123,-3.206432,3.955444,6634566447,6634566443,20.980
6,Bruntsfield,55.937159,-3.206435,Bruntsfield Links,55.937123,-3.206432,3.955444,6634566447,6634566443,20.980
9,St. John's Road 1,55.942746,-3.281623,St. John's Road 2,55.942945,-3.290794,573.479672,2267137214,606216,568.310
11,Sighthill - Edinburgh College,55.926684,-3.289481,St. John's Road 2,55.942945,-3.290794,1812.321621,3030358473,606216,2593.979
...,...,...,...,...,...,...,...,...,...,...
272,Meadows East,55.939809,-3.182739,Gladstone Terrace,55.937963,-3.185021,250.189667,6629509025,14348045,366.003
273,Waverley Court,55.951734,-3.184179,St Andrews House,55.953164,-3.181682,222.845987,524066682,259849803,354.904
274,Scotstoun House,55.981107,-3.394211,Tesco Ferrymuir,55.983766,-3.401352,535.114797,25650235,3885082380,727.296
276,Charlotte Square,55.952335,-3.207101,Castle Street,55.951501,-3.203184,261.766730,3061162846,626129,391.098


In [ ]:
df3